In [9]:
#!pip install datetime
#!pip install numpy
#!pip install pandas
#!pip install xgboost
#!pip install scikit-learn
#!pip install agentds-bench
#!pip install imblearn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 8.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import xgboost
import sklearn
from itertools import product
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

In [141]:
X_df = X_df.fillna(-1)
X = X_df.to_numpy()
y = Y_df.to_numpy()
# define pipeline
over = SMOTE(sampling_strategy=0.3)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
#X, y = pipeline.fit_resample(X, y)
print(y[0:2])
print(X[0:2, :])

[[0]
 [0]]
[[0 1 1 44.41617907 0 300.342310530943 -1.0 -1 -1.0 -1.0
  53.84044406467309 7.325337186630032 781.4387832676174 49665.18770258372
  2 0 0 0 0 0 0 0 9 False -1.0 -1.0 -1.0 56 -1.0 0 0 -1.0 -1.0 -1.0 0]
 [0 1 2 68.53060371 0 300.342310530943 -1.0 -1 -1.0 -1.0
  53.84044406467309 7.325337186630032 781.4387832676174 49665.18770258372
  2 0 0 0 0 0 0 2 3 False -1.0 -1.0 -1.0 57 -1.0 24 0 44.41617907
  1.5429198356300664 0.5429198356300664 0]]


In [68]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [69]:
# Train the model

neg_count = np.sum(y_train == 0)
pos_count = np.sum(y_train == 1)
scale_pos_weight_value = np.sqrt(neg_count / pos_count) # square root can be used if scale_pos_weight is too high

params = {'objective':'binary:logistic',
          'eval_metric':'aucpr',
          'random_state':42,
          'n_estimators':1000,
          'subsample': 0.8,
          'colsample_bytree': 0.8,
          'reg_lambda':1.5,
          'eta': 0.05
            }
xgb = XGBClassifier(**params)

# Cross validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
macro_f1_scorer = make_scorer(f1_score, average='macro')
param_grid = {
    'max_depth': [4, 5, 6],
    'min_child_weight': [1, 3, 5],
    #'subsample': [0.6, 0.8, 1.0],
    #'colsample_bytree': [0.6, 0.8, 1.0],
    #'eta': [0.01, 0.05, 0.1, 0.2],
    'scale_pos_weight': [scale_pos_weight_value, 50, 100]
}
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=cv,
    scoring=macro_f1_scorer,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
best_xgb_model = grid_search.best_estimator_

preds = best_xgb_model.predict(X_test)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best parameters found: {'max_depth': 6, 'min_child_weight': 3, 'scale_pos_weight': 50}
Best cross-validation score: 0.6966


In [142]:
best_params = grid_search.best_params_
#best_params.update({'min_child_weight': 1})
#best_params.update({'scale_pos_weight':scale_pos_weight_value})
best_params.update({'max_depth':6})
final_xgb = XGBClassifier(
    **best_params,
    n_estimators=1000,
    eval_metric='aucpr',
    colsample_bytree = 0.6,
    eta = 0.05,
    subsample = 0.5,
    gamma=20
)
final_xgb.fit(
    X,
    y,
    #X_train,
    #y_train,
    verbose=False
)
#final_xgb.save_model('xgboost_model.json')
#best_xgb_model = XGBClassifier()
#best_xgb_model.load_model('xgboost_model.json')
#print(best_xgb_model.get_params())

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.6
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'aucpr'


In [138]:
y_test = y_test.reshape(-1)
preds = final_xgb.predict(X_test)
F1 = f1_score(y_test, preds, average='macro')
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
print("Test Accuracy: ", (preds == y_test).sum()/len(y_test))
print("Test Macro F1-Score: ", F1)
print("Test Precision: ", precision)
print("Test Recall: ", recall)

y_train = y_train.reshape(-1)
preds = final_xgb.predict(X_train)
F1 = f1_score(y_train, preds, average='macro')
precision = precision_score(y_train, preds)
recall = recall_score(y_train, preds)
print("Train Accuracy: ", (preds == y_train).sum()/len(y_train))
print("Train Macro F1-Score: ", F1)
print("Train Precision: ", precision)
print("Train Recall: ", recall)

Test Accuracy:  0.996815809342481
Test Macro F1-Score:  0.6552304745770029
Test Precision:  0.7586206896551724
Test Recall:  0.19642857142857142
Train Accuracy:  0.9997045596297147
Train Macro F1-Score:  0.977255873292002
Train Precision:  0.949874686716792
Train Recall:  0.959493670886076


In [ ]:
#params = {'objective': 'binary:logistic', 'base_score': [0.0032416375],
          #'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': None,
          #'colsample_bynode': None, 'colsample_bytree': None, 'device': None,
          #'early_stopping_rounds': None, 'enable_categorical': False,
          #'eval_metric': None, 'feature_types': None, 'feature_weights': None,
          #'gamma': None, 'grow_policy': None, 'importance_type': None,
          #'interaction_constraints': None, 'learning_rate': None,
          #'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None,
          #'max_delta_step': None, 'max_depth': None, 'max_leaves': None,
          #'min_child_weight': None, 'monotone_constraints': None,
          #'multi_strategy': None, 'n_estimators': 2000, 'n_jobs': None,
          #'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None,
          #'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': scale_pos_weight_value,
          #subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
#model = XGBClassifier(**params)
#model.fit(X_train, y_train)

In [143]:
df = pd.read_csv('final_test.csv')
X_df = df[labels]
X = X_df.to_numpy()

In [144]:
# Make predictions
predictions = final_xgb.predict(X)
#predictions = model.predict(X)

# Create submission file (format: TxnID,FraudLabel)
submission_df = pd.DataFrame({
    'TxnID': df['TxnID'],
    'FraudLabel': predictions
})

# Save predictions
submission_df.to_csv("retailbanking_challenge1_predictions.csv", index=False)
print(len(X))
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Fraud rate: {predictions.mean():.3f} ({predictions.sum()} fraud cases out of {len(predictions)})")

151587
✅ Predictions saved: 151587 predictions
   Preview:                                   TxnID  FraudLabel
0  c77cfd37-2a4f-41a2-ae45-696aaa2ec4e9           0
1  1e3da79d-891c-4769-af85-4b9bfc2aab7c           0
2  b1e5643f-3fee-41e6-b003-a360810dfa3d           0
   Fraud rate: 0.001 (83 fraud cases out of 151587)


In [145]:
from agentds import BenchmarkClient

client = BenchmarkClient(
    api_key="adsb_hdm1DRk1iW2I1VA84Oc9jz6z_1756090832",        # Get from your team dashboard
    team_name="agi"     # Your exact team name
)

# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Retailbanking", 1, "retailbanking_challenge1_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Retail Banking Challenge 2!")


🚀 Submitting predictions...
✅ Prediction submitted successfully!
📊 Score: 0.4987 (Macro-F1)
✅ Validation passed
✅ Submission successful!
   📊 Score: 0.4987
   📏 Metric: Macro-F1
   ✔️  Validation: Passed

🎯 Next steps:
   1. Try incorporating relevant information outside this table!
   2. Move on to Retail Banking Challenge 2!
